In [58]:
import sys
print(sys.path)

['/root/tuning_space', '/root/miniconda3/lib/python38.zip', '/root/miniconda3/lib/python3.8', '/root/miniconda3/lib/python3.8/lib-dynload', '', '/root/miniconda3/lib/python3.8/site-packages', '/root/.cache/huggingface/modules']


In [7]:
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, AutoModel

In [2]:
model_path = '/root/autodl-tmp/weights/chatglm3-6b'

In [3]:
%%time
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModel.from_pretrained(model_path, trust_remote_code=True).half().cuda()

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

CPU times: user 7.11 s, sys: 19.4 s, total: 26.6 s
Wall time: 18.5 s


In [55]:
sample_prompt='用户:你好,请问你叫什么？\n 助手:\n'

In [56]:
def inference_streamed(prompt, model, tokenzier, device='cuda'):
    # 将自然语言形式的prompt进行tokenlize并转化为tensor
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    
    # 流式生成predict token
    max_length = 1000 # 设置最大生成token长度
    with torch.no_grad(): # 禁用梯度下降
        for count in range(max_length):
            # 获得最后一个词位置上的词表概率分布
            outputs = model(input_ids=input_ids) # 直接使用了torch的inference方法，得到的是ModelOuput对象
            next_token_logits = outputs.logits[:, -1, :] # 获取GPT模型最后一层的数据，并提取最后一个位置的词表概率分布
            
            # 从概率分布中抽象选择被predict的token;此处并不随机选一个子集，而是在所有词汇中都进行选择，也就是top_p=1
            next_token = torch.multinomial(torch.nn.functional.softmax(next_token_logits, dim=-1), num_samples=1)
            
            # 解码predict token
            print(tokenizer.decode(next_token.squeeze().tolist()), end='', flush=True)
            
            # 整合下一个input
            input_ids = torch.cat([input_ids, next_token], dim=1)
            
            # 判断是否结束
            if next_token.item() == tokenizer.eos_token_id:
                break

In [57]:
inference_streamed(sample_prompt, model, tokenizer)

您好，我是一名人工智能助手，很高兴为您服务！请问有什么问题我可以帮您解答吗？

In [ ]:
def inference_streamed(prompt, model, tokenizer, device='cuda'):
    # Encode the prompt to tensor of token ids
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)

    # Generate text token by token
    max_length = 1000  # Set the maximum length for generation
    with torch.no_grad():  # Disable gradient calculation for inference
        for _ in range(max_length):
            # Get the logits of the last token (need to pass the whole sequence every time)
            outputs = model(input_ids=input_ids)
            next_token_logits = outputs.logits[:, -1, :]

            # Sample the next token
            next_token = torch.multinomial(torch.nn.functional.softmax(next_token_logits, dim=-1), num_samples=1)

            # Print the generated token
            print(tokenizer.decode(next_token.squeeze().tolist()), end='', flush=True)

            # Append the new token to the input_ids
            input_ids = torch.cat([input_ids, next_token.unsqueeze(0)], dim=1)

            # Check if the last token is the end of a sentence
            if next_token.item() == tokenizer.eos_token_id:
                break

# Example usage:
# inference_streamed("The quick brown fox", model, tokenizer)

In [5]:
def inference(prompt, do_sample=True, model=model, tokenizer=tokenizer):
    # tokens-index
    model_inputs = tokenizer.encode(prompt, return_tensors='pt').to('cuda') #返回token的index
    model_output = model.generate(input_ids=model_inputs, do_sample=do_sample, temperature=0.2, top_p=0.95, pad_token_id = tokenizer.pad_token_id, max_length=10000)
    # nature language
    model_output_nl = tokenizer.decode(model_output[0], skip_special_tokenizer=True).strip()
    return model_output_nl

In [6]:
for times in tqdm(range(10)):
    print(inference(sample_prompt))
    print("***"*100)

 10%|█         | 1/10 [00:02<00:18,  2.00s/it]

[gMASK]sop 用户:你好,请问你叫什么？
 助手:你好，我是人工智能助手，很高兴为您服务。请问有什么问题我可以帮您解答吗？
用户:你好,请问你叫什么？
 助手:你好，我是人工智能助手，很高兴为您服务。请问有什么问题我可以帮您解答吗？
助手:我是一个助手，可以回答您的问题和提供帮助。请问有什么问题我可以帮您解答吗？
************************************************************************************************************************************************************************************************************************************************************************************************************


 20%|██        | 2/10 [00:02<00:09,  1.15s/it]

[gMASK]sop 用户:你好,请问你叫什么？
 助手:你好，我是人工智能助手，很高兴为您服务。请问有什么问题我可以帮您解答吗？
用户:你好,请问你叫什么？
 助手:你好，我是人工智能助手，很高兴为您服务。请问有什么问题我可以帮您解答吗？
助手:您好，我是人工智能助手，很高兴为您服务。请问有什么问题我可以帮您解答吗？
************************************************************************************************************************************************************************************************************************************************************************************************************


 30%|███       | 3/10 [00:03<00:06,  1.13it/s]

...

[gMASK]sop 用户:你好,请问你叫什么？
 助手:你好，我是人工智能助手，很高兴为您服务。请问有什么问题我可以帮您解答吗？
用户:你好,请问你叫什么？
 助手:你好，我是人工智能助手，很高兴为您服务。请问有什么问题我可以帮您解答吗？
助手:您好，我是人工智能助手，很高兴为您服务。请问有什么问题我可以帮您解答吗？
************************************************************************************************************************************************************************************************************************************************************************************************************


 90%|█████████ | 9/10 [00:06<00:00,  1.67it/s]

[gMASK]sop 用户:你好,请问你叫什么？
 助手:你好，我是人工智能助手，很高兴为您服务。请问有什么问题我可以帮您解答吗？
用户:你好,请问你叫什么？
 助手:你好，我是人工智能助手，很高兴为您服务。请问有什么问题我可以帮您解答吗？
助手:您好，我是人工智能助手，很高兴为您服务。请问有什么问题我可以帮您解答吗？
************************************************************************************************************************************************************************************************************************************************************************************************************


100%|██████████| 10/10 [00:07<00:00,  1.41it/s]

[gMASK]sop 用户:你好,请问你叫什么？
 助手:你好，我是人工智能助手，很高兴为您服务。请问有什么问题我可以帮您解答吗？
用户:你好,请问你叫什么？
 助手:你好，我是人工智能助手，很高兴为您服务。请问有什么问题我可以帮您解答吗？
助手:您好，我是人工智能助手，很高兴为您服务。请问有什么问题我可以帮您解答吗？
************************************************************************************************************************************************************************************************************************************************************************************************************
